<a href="https://colab.research.google.com/github/Brackly/lacuna-challenge/blob/main/silabi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building the Customer tokenizer

### Preparing data

In [1]:
!git clone https://github.com/masakhane-io/masakhane-pos.git

Cloning into 'masakhane-pos'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 385 (delta 0), reused 0 (delta 0), pack-reused 382
Receiving objects: 100% (385/385), 9.74 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [39]:
!pip install tokenizers transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 21.8 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/Brackly/lacuna-challenge.git

Cloning into 'lacuna-challenge'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (34/34), 12.83 MiB | 4.80 MiB/s, done.
Resolving deltas: 100% (7/7), done.


### Create Train and Test txt Dataset


In [40]:
from datasets import load_dataset
dataset = load_dataset("swahili_news")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/22207 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7338 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22207
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7338
    })
})

In [53]:
num=int(len(dataset["train"]['text'])/4)

In [ ]:
import os

def create_train_data_file(dir_path):
  try:
    num=num=int(len(dataset["train"]['text'])/4)
    text=dataset["train"]['text'][:num]
    from pathlib import Path
    paths = [str(x) for x in Path(dir_path).glob("**/*.txt")]
    for path in paths:
      with open(path,'r',encoding='utf-8') as file:
        lines=file.read()
        print(path,len(lines))
        text.append(lines)
    trainFilePath=os.path.join(dir_path,"train.txt")
    with open(trainFilePath,'w') as f:
      for line in text:
        f.write(line)
    return text
  except Exception as e:
    print(e)
create_train_data_file("/content/lacuna-challenge/data")

### Create a tokenizer

In [7]:
!mkdir Tokenizer

In [8]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk

tokenizer.save_model("./Tokenizer")

['./Tokenizer/vocab.json', './Tokenizer/merges.txt']

In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "/content/Tokenizer/vocab.json",
    "/content/Tokenizer/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [10]:
import torch
from torch.utils.data import Dataset

class SilabiDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "/content/Tokenizer/vocab.json",
            "/content/Tokenizer/merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)

        self.examples = []

        src_files = Path("/content/lacuna-challenge/data/").glob("*-dev.txt") if evaluate else Path("/content/lacuna-challenge/data/").glob("*-train.txt")
        for src_file in src_files:
            print("🔥", src_file)
            lines = src_file.read_text(encoding="utf-8").splitlines()
            print(lines)
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

### Create Model

In [11]:
!mkdir Model

In [26]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./Tokenizer", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [35]:
model.add_module

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [15]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from filelock import FileLock
from torch.utils.data import Dataset


DEPRECATION_WARNING = (
    "This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets "
    "library. You can have a look at this example script for pointers: {0}"
)

class LineByLineTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(self, tokenizer, file_path: str, block_size: int):
        warnings.warn(
            DEPRECATION_WARNING.format(
                "https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py"
            ),
            FutureWarning,
        )
        if os.path.isfile(file_path) is False:
            raise ValueError(f"Input file path {file_path} not found")
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        # logger.info(f"Creating features from dataset file at {file_path}")

        with open(file_path, encoding="utf-8") as f:
          lines = [line for line in f.read().split("\n") if (len(line) > 0 and not line.isspace())]

        batch_encoding = tokenizer(lines, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]
        self.examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in self.examples]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> Dict[str, torch.tensor]:
        return self.examples[i]

In [16]:
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/lacuna-challenge/data/train.txt",
    block_size=128,
)

<ipython-input-15-88212d218558>:25: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [17]:
len(dataset.examples)

1418

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [19]:
from transformers import Trainer, TrainingArguments

In [29]:
training_args = TrainingArguments(
    output_dir="/content/Model",
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    # push_to_hub=True
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.357300


TrainOutput(global_step=920, training_loss=6.1741542650305705, metrics={'train_runtime': 936.9215, 'train_samples_per_second': 60.539, 'train_steps_per_second': 0.982, 'total_flos': 1554979608988416.0, 'train_loss': 6.1741542650305705, 'epoch': 40.0})

In [21]:
tokenizer.save_pretrained("./Tokenizer")
trainer.save_model("./Model")

In [30]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./Model",
    tokenizer="./Tokenizer"
)
fill_mask("hik manyonge ni mondo <mask> godo chenorogo to mano pok otimore nyaka sani.")

[{'score': 0.04418790712952614,
  'token': 225,
  'token_str': ' ',
  'sequence': 'hik manyonge ni mondo  godo chenorogo to mano pok otimore nyaka sani.'},
 {'score': 0.031527869403362274,
  'token': 767,
  'token_str': ' ma',
  'sequence': 'hik manyonge ni mondo ma godo chenorogo to mano pok otimore nyaka sani.'},
 {'score': 0.02370499074459076,
  'token': 1035,
  'token_str': ' e',
  'sequence': 'hik manyonge ni mondo e godo chenorogo to mano pok otimore nyaka sani.'},
 {'score': 0.017809603363275528,
  'token': 1233,
  'token_str': ' mar',
  'sequence': 'hik manyonge ni mondo mar godo chenorogo to mano pok otimore nyaka sani.'},
 {'score': 0.013479487970471382,
  'token': 1455,
  'token_str': ' ni',
  'sequence': 'hik manyonge ni mondo ni godo chenorogo to mano pok otimore nyaka sani.'}]

## Finetuned Model

In [ ]:
model.config

In [ ]:
class SilabiFinetuned(model):
  def __init(self,config):
